In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter
from scipy.stats import norm
%matplotlib notebook

# ---------------------------
# Data and Simulation Functions
# ---------------------------
def compute_ground_truth(x, mean=5, std=1):
    """
    Compute the ground truth PDF for x using a normal distribution.
    
    Args:
        x (np.ndarray): 1D array of x-values.
        mean (float): Mean of the real distribution.
        std (float): Standard deviation of the real distribution.
        
    Returns:
        np.ndarray: Ground truth PDF values.
    """
    return norm.pdf(x, mean, std)

def compute_generator_pdf(x, frame, total_frames, init_mean=2, init_std=2, target_mean=5, target_std=1):
    """
    Compute the generator's PDF for the given frame by linearly interpolating 
    its parameters from initial to target values. This makes the generated data 
    converge to the ground truth.
    
    Args:
        x (np.ndarray): 1D array of x-values.
        frame (int): Current frame (epoch) index.
        total_frames (int): Total number of frames.
        init_mean (float): Initial mean of the generator's distribution.
        init_std (float): Initial standard deviation of the generator's distribution.
        target_mean (float): Target mean (should match ground truth).
        target_std (float): Target standard deviation (should match ground truth).
        
    Returns:
        np.ndarray: Generator PDF values for this frame.
    """
    t = frame / (total_frames - 1)  # t varies from 0 to 1
    gen_mean = init_mean + (target_mean - init_mean) * t
    gen_std = init_std + (target_std - init_std) * t
    return norm.pdf(x, gen_mean, gen_std)

def compute_discriminator_prob(real_pdf, gen_pdf, factor=10):
    """
    Simulate the discriminator's output as a logistic function of the difference
    between generator and real PDFs.
    
    Args:
        real_pdf (np.ndarray): Ground truth PDF values.
        gen_pdf (np.ndarray): Generator PDF values.
        factor (float): Scaling factor for the logistic function.
        
    Returns:
        np.ndarray: Discriminator probability outputs.
    """
    return 1 / (1 + np.exp(-(gen_pdf - real_pdf) * factor))

def simulate_training(x, num_frames):
    """
    Simulate the training process over a specified number of frames.
    The generated PDF converges linearly to the ground truth PDF.
    
    Args:
        x (np.ndarray): 1D array of x-values.
        num_frames (int): Number of simulation frames.
        
    Returns:
        tuple: (ground_truth_data, generator_data, discriminator_data)
               Each is a NumPy array of shape (num_frames, len(x)).
    """
    ground_truth_list = []
    generator_list = []
    discriminator_list = []
    
    # Ground truth PDF remains constant throughout the simulation
    real_pdf = compute_ground_truth(x)
    
    for frame in range(num_frames):
        gen_pdf = compute_generator_pdf(x, frame, num_frames)
        disc_pdf = compute_discriminator_prob(real_pdf, gen_pdf)
        
        ground_truth_list.append(real_pdf)
        generator_list.append(gen_pdf)
        discriminator_list.append(disc_pdf)
    
    return (np.array(ground_truth_list), 
            np.array(generator_list), 
            np.array(discriminator_list))

# ---------------------------
# Animation Functions
# ---------------------------
def create_animation(x, ground_truth_data, generator_data, discriminator_data, num_frames, interval=200):
    """
    Create an animation using precomputed data.
    
    Args:
        x (np.ndarray): 1D array of x-values.
        ground_truth_data (np.ndarray): Array of ground truth PDFs (num_frames x len(x)).
        generator_data (np.ndarray): Array of generator PDFs (num_frames x len(x)).
        discriminator_data (np.ndarray): Array of discriminator outputs (num_frames x len(x)).
        num_frames (int): Number of frames.
        interval (int): Delay between frames in milliseconds.
        
    Returns:
        tuple: (ani, fig) where ani is the FuncAnimation object and fig is the figure.
    """
    fig, ax = plt.subplots()
    ax.set_xlim(np.min(x), np.max(x))
    ax.set_ylim(0, 0.8)
    ax.set_title("Training Discriminator D")
    ax.set_xlabel("x")
    ax.set_ylabel("Probability")
    
    real_line, = ax.plot([], [], 'k--', label='Real Data (pdata)')
    fake_line, = ax.plot([], [], 'g-', label='Generator (pg)')
    disc_line, = ax.plot([], [], 'b--', label='Discriminator (D)')
    ax.legend()
    
    def update(frame):
        real_line.set_data(x, ground_truth_data[frame])
        fake_line.set_data(x, generator_data[frame])
        disc_line.set_data(x, discriminator_data[frame])
        return real_line, fake_line, disc_line
    
    ani = animation.FuncAnimation(fig, update, frames=num_frames, interval=interval, blit=True)
    return ani, fig

def save_animation(ani, mp4_filename, gif_filename, fps=10):
    """
    Save the animation in both MP4 and GIF formats.
    
    Args:
        ani (FuncAnimation): The animation object.
        mp4_filename (str): Output filename for MP4.
        gif_filename (str): Output filename for GIF.
        fps (int): Frames per second.
    """
    writer = FFMpegWriter(fps=fps, metadata=dict(artist='Your Name'),
                           extra_args=['-vcodec', 'libx264'])
    ani.save(mp4_filename, writer=writer)
    ani.save(gif_filename, writer='pillow', fps=fps)

# ---------------------------
# Direct Function Calls (No main function)
# ---------------------------

# Define x-range and simulation parameters
x = np.linspace(-2, 12, 100)
num_frames = 30

# Simulate training: all data is stored in arrays
ground_truth_data, generator_data, discriminator_data = simulate_training(x, num_frames)
print("Ground truth data shape:", ground_truth_data.shape)
print("Generator data shape:", generator_data.shape)
print("Discriminator data shape:", discriminator_data.shape)

# Create and display the animation
ani, fig = create_animation(x, ground_truth_data, generator_data, discriminator_data, num_frames)
plt.show()

# Save the animation in MP4 and GIF formats
save_animation(ani, "gan_training_progress.mp4", "gan_training_progress.gif", fps=10)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter
from scipy.stats import norm
%matplotlib notebook

# Hyperparameters
latent_dim = 10       # Dimensionality of the latent (noise) vector
hidden_dim = 128      # Number of neurons in the hidden layers
data_dim = 1          # Output dimensionality (univariate distribution)
batch_size = 128      # Number of samples per training batch
epochs = 200          # Number of training epochs
lr = 0.0002           # Learning rate for the optimizers

def true_distribution(n):
    """
    Generate samples from the true (real) distribution.
    
    Args:
        n (int): Number of samples to generate.
        
    Returns:
        np.ndarray: Array of shape (n, 1) with samples drawn from a normal 
                    distribution (mean=5, std=1).
    """
    mean = 5
    std = 1
    return np.random.normal(mean, std, n).reshape(-1, 1)

def build_generator():
    """
    Build the generator model using a functional approach.
    
    Returns:
        torch.nn.Sequential: The generator model mapping latent vectors to data.
    """
    model = nn.Sequential(
        nn.Linear(latent_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, data_dim),
    )
    return model

def train_generator(generator, optimizer):
    """
    Perform a single training step for the generator.
    
    Args:
        generator (torch.nn.Module): The generator model.
        optimizer (torch.optim.Optimizer): Optimizer for updating generator parameters.
        
    Returns:
        np.ndarray: The fake data generated during this training step.
    """
    optimizer.zero_grad()
    # Sample random latent vectors from a normal distribution
    z = torch.randn(batch_size, latent_dim)
    # Generate fake data from the latent vectors
    fake_data = generator(z)
    # Define a simple loss as the negative mean of the generator output
    loss = -torch.mean(fake_data)
    loss.backward()
    optimizer.step()
    # Detach generated data from the computation graph and convert to NumPy array
    return fake_data.detach().numpy()

def update(frame, x, real_pdf, fake_line, disc_line):
    """
    Update function for the animation.
    
    Simulates the evolution of the generator's output and a discriminator's response.
    
    Args:
        frame (int): Current frame index in the animation.
        x (np.ndarray): Array of x-values for plotting.
        real_pdf (np.ndarray): The probability density of the real distribution.
        fake_line (matplotlib.lines.Line2D): Line object for the generator's output.
        disc_line (matplotlib.lines.Line2D): Line object for the discriminator's output.
    
    Returns:
        tuple: Updated fake_line and disc_line for the animation.
    """
    # Simulate improvement in the generator's output over time:
    gen_mean = 2 + frame * 0.1
    gen_std = max(2 - frame * 0.05, 0.5)  # Prevent standard deviation from dropping too low
    # Compute the generator's PDF with current parameters
    gen_pdf = norm.pdf(x, gen_mean, gen_std)
    
    # Simulate a discriminator response using a logistic function on the difference
    disc_prob = 1 / (1 + np.exp(-(gen_pdf - real_pdf) * 10))
    
    # Update the plot data for the generator and discriminator lines
    fake_line.set_data(x, gen_pdf)
    disc_line.set_data(x, disc_prob)
    
    return fake_line, disc_line

# ---------------------------
# Direct function calls below
# ---------------------------

# Build generator models using the functional approach
generator1 = build_generator()
generator2 = build_generator()

# Initialize optimizers for each generator
optimizer1 = optim.Adam(generator1.parameters(), lr=lr)
optimizer2 = optim.Adam(generator2.parameters(), lr=lr)

# Save ground truth data by sampling from the true distribution
num_ground_truth_samples = 1000
ground_truth_data = true_distribution(num_ground_truth_samples)

# Initialize lists to store generated data from each generator
generated_data_gen1 = []
generated_data_gen2 = []

# Training loop: generate fake data and save it in array objects
for epoch in range(epochs):
    fake_data1 = train_generator(generator1, optimizer1)
    fake_data2 = train_generator(generator2, optimizer2)
    generated_data_gen1.append(fake_data1)
    generated_data_gen2.append(fake_data2)

# ---------------------------
# Setup for visualization/animation
# ---------------------------
fig, ax = plt.subplots()
ax.set_xlim(0, 10)
ax.set_ylim(0, 0.8)
ax.set_title("Training Discriminator Simulation")
ax.set_xlabel("x")
ax.set_ylabel("Probability")

# Create x values and compute the real probability density function (PDF)
x = np.linspace(0, 10, 1000)
real_pdf = norm.pdf(x, 5, 1)
# Plot the real data distribution (dashed black line)
ax.plot(x, real_pdf, 'k--', label='Real Data (pdata)')

# Initialize empty plot lines for generator output and discriminator response
fake_data_line, = ax.plot([], [], 'g-', label='Generator (pg)')
discriminator_line, = ax.plot([], [], 'b--', label='Discriminator (D)')
ax.legend()

# Create animation; using a lambda to pass extra parameters to the update function
ani = animation.FuncAnimation(
    fig,
    lambda frame: update(frame, x, real_pdf, fake_data_line, discriminator_line),
    frames=30,
    interval=200,
    blit=True
)

# ---------------------------
# Save the animation in MP4 and GIF formats
# ---------------------------
# Create an FFMpegWriter object for MP4 saving
mp4_writer = FFMpegWriter(
    fps=10,
    metadata=dict(artist='Your Name'),
    extra_args=['-vcodec', 'libx264']
)
ani.save('gan_training_progress.mp4', writer=mp4_writer)

# Save as GIF using the pillow writer
ani.save('gan_training_progress.gif', writer='pillow', fps=10)
#
plt.show()


<IPython.core.display.Javascript object>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter
from scipy.stats import norm

# ---------------------------
# Hyperparameters
# ---------------------------
latent_dim = 10       # Dimensionality of the latent (noise) vector
hidden_dim = 128      # Number of neurons in the hidden layers
data_dim = 1          # Output dimensionality (univariate distribution)
batch_size = 128      # Number of samples per training batch
epochs = 200          # Number of training epochs
lr = 0.0002           # Learning rate for the optimizers

# ---------------------------
# Data Function
# ---------------------------
def true_distribution(n):
    """
    Generate n samples from the real distribution (N(5,1)).
    
    Args:
        n (int): Number of samples.
    
    Returns:
        np.ndarray: Samples of shape (n, 1).
    """
    mean = 5
    std = 1
    return np.random.normal(mean, std, n).reshape(-1, 1)

# ---------------------------
# Generator Model Function (using nn.Sequential)
# ---------------------------
def build_generator():
    """
    Build the generator model.
    
    Returns:
        torch.nn.Sequential: A simple feed-forward network.
    """
    return nn.Sequential(
        nn.Linear(latent_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, data_dim)
    )

# ---------------------------
# Training Function
# ---------------------------
def train_generator(generator, optimizer):
    """
    Train the generator for one step. Instead of the old loss (negative mean),
    this convergence loss encourages the generator's batch statistics (mean and std)
    to match the ground truth (mean=5, std=1).
    
    Args:
        generator (torch.nn.Module): The generator model.
        optimizer (torch.optim.Optimizer): The optimizer.
    
    Returns:
        tuple: (loss_value, fake_data, sample_mean, sample_std)
    """
    optimizer.zero_grad()
    z = torch.randn(batch_size, latent_dim)
    fake_data = generator(z)
    sample_mean = torch.mean(fake_data)
    sample_std = torch.std(fake_data)
    # Convergence loss: MSE for mean and std
    loss = (sample_mean - 5)**2 + (sample_std - 1)**2
    loss.backward()
    optimizer.step()
    return loss.item(), fake_data.detach().numpy(), sample_mean.item(), sample_std.item()

# ---------------------------
# Training Loop and Data Storage
# ---------------------------
# Build two generator models and corresponding optimizers.
generator1 = build_generator()
generator2 = build_generator()
optimizer1 = optim.Adam(generator1.parameters(), lr=lr)
optimizer2 = optim.Adam(generator2.parameters(), lr=lr)

# Lists for storing loss and generator statistics for animation.
losses1 = []
losses2 = []
stats1 = []  # Each element is a tuple: (mean, std) for generator1
stats2 = []  # For generator2

# Train both generators and print progress.
for epoch in range(epochs):
    loss1, fake1, mean1, std1 = train_generator(generator1, optimizer1)
    loss2, fake2, mean2, std2 = train_generator(generator2, optimizer2)
    losses1.append(loss1)
    losses2.append(loss2)
    stats1.append((mean1, std1))
    stats2.append((mean2, std2))
    if epoch == 0 or (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:03d}/{epochs} | Loss1: {loss1:.4f} | Loss2: {loss2:.4f} | "
              f"Mean1: {mean1:.4f}, Std1: {std1:.4f} | Mean2: {mean2:.4f}, Std2: {std2:.4f}")

# ---------------------------
# Compute PDFs for Animation
# ---------------------------
# Define x-axis for plotting the PDFs.
x = np.linspace(0, 10, 1000)
# The real PDF (ground truth) is constant.
real_pdf = norm.pdf(x, 5, 1)
# Compute the generator PDFs at each epoch from stored statistics.
gen_pdf1 = [norm.pdf(x, m, s) for m, s in stats1]
gen_pdf2 = [norm.pdf(x, m, s) for m, s in stats2]
gen_pdf1 = np.array(gen_pdf1)
gen_pdf2 = np.array(gen_pdf2)

# ---------------------------
# Animation Functions
# ---------------------------
def create_animation(x, real_pdf, gen_pdf1, gen_pdf2, losses1, losses2, epochs, interval=100):
    """
    Create an animation showing the evolution of the generator PDFs and loss.
    
    Args:
        x (np.ndarray): x-axis values.
        real_pdf (np.ndarray): Real PDF values.
        gen_pdf1 (np.ndarray): Generator 1 PDF for each epoch.
        gen_pdf2 (np.ndarray): Generator 2 PDF for each epoch.
        losses1 (list): Loss values for generator1.
        losses2 (list): Loss values for generator2.
        epochs (int): Total number of epochs (frames).
        interval (int): Delay between frames (ms).
    
    Returns:
        tuple: (ani, fig) the animation object and the figure.
    """
    fig, ax = plt.subplots()
    ax.set_xlim(np.min(x), np.max(x))
    ax.set_ylim(0, 0.8)
    ax.set_title("Training Progress: Generator Convergence")
    ax.set_xlabel("x")
    ax.set_ylabel("Probability")
    
    # Plot the fixed real data PDF.
    ax.plot(x, real_pdf, 'k--', label="Real Data (pdata)")
    line1, = ax.plot([], [], 'g-', label="Generator 1 (pg)")
    line2, = ax.plot([], [], 'b-', label="Generator 2 (pg)")
    
    # Text annotation for loss information.
    loss_text = ax.text(0.05, 0.75, "", transform=ax.transAxes)
    ax.legend()
    
    def update(frame):
        line1.set_data(x, gen_pdf1[frame])
        line2.set_data(x, gen_pdf2[frame])
        loss_text.set_text(f"Epoch: {frame+1}\nLoss1: {losses1[frame]:.4f}\nLoss2: {losses2[frame]:.4f}")
        return line1, line2, loss_text
    
    ani = animation.FuncAnimation(fig, update, frames=epochs, interval=interval, blit=True)
    return ani, fig

# Create the animation.
ani, fig = create_animation(x, real_pdf, gen_pdf1, gen_pdf2, losses1, losses2, epochs, interval=100)
plt.show()

# ---------------------------
# Save the Animation
# ---------------------------
def save_animation(ani, mp4_filename, gif_filename, fps=10):
    """
    Save the animation in both MP4 and GIF formats.
    
    Args:
        ani (FuncAnimation): The animation object.
        mp4_filename (str): Output file name for MP4.
        gif_filename (str): Output file name for GIF.
        fps (int): Frames per second.
    """
    writer = FFMpegWriter(fps=fps, metadata=dict(artist='Your Name'),
                           extra_args=['-vcodec', 'libx264'])
    ani.save(mp4_filename, writer=writer)
    ani.save(gif_filename, writer='pillow', fps=fps)

# Save the animation.
save_animation(ani, "gan_training_progress.mp4", "gan_training_progress.gif", fps=10)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter
from scipy.stats import norm

# ---------------------------
# Hyperparameters
# ---------------------------
latent_dim = 10       # Dimensionality of latent vector
hidden_dim = 128      # Hidden layer size
data_dim = 1          # Output dimension (univariate)
batch_size = 128      # Batch size for training
epochs = 200          # Number of training epochs
lr = 0.0002           # Learning rate

# ---------------------------
# Data Function
# ---------------------------
def true_distribution(n):
    """
    Generate n samples from N(5, 1).
    
    Args:
        n (int): Number of samples.
    
    Returns:
        np.ndarray: Samples of shape (n, 1).
    """
    mean, std = 5, 1
    return np.random.normal(mean, std, n).reshape(-1, 1)

# ---------------------------
# Model Building Functions (using nn.Sequential)
# ---------------------------
def build_generator():
    """
    Build a generator model.
    
    Returns:
        torch.nn.Sequential: A feed-forward network mapping latent vectors to data.
    """
    return nn.Sequential(
        nn.Linear(latent_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, data_dim)
    )

def build_discriminator():
    """
    Build a discriminator model.
    
    Returns:
        torch.nn.Sequential: A feed-forward network mapping data to a probability.
    """
    return nn.Sequential(
        nn.Linear(data_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, 1),
        nn.Sigmoid()
    )

# ---------------------------
# Training Step Functions
# ---------------------------
def train_discriminator(discriminator, generator, D_optimizer, criterion, real_data):
    """
    Train the discriminator for one step.
    
    Args:
        discriminator (torch.nn.Module): The discriminator model.
        generator (torch.nn.Module): The generator model.
        D_optimizer (torch.optim.Optimizer): Optimizer for the discriminator.
        criterion: Loss function (BCE).
        real_data (torch.Tensor): Batch of real data.
    
    Returns:
        float: Discriminator loss.
    """
    D_optimizer.zero_grad()
    # Real samples and labels
    real_labels = torch.ones(real_data.size(0), 1)
    real_preds = discriminator(real_data)
    loss_real = criterion(real_preds, real_labels)
    
    # Fake samples
    z = torch.randn(real_data.size(0), latent_dim)
    fake_data = generator(z)
    fake_labels = torch.zeros(real_data.size(0), 1)
    fake_preds = discriminator(fake_data.detach())
    loss_fake = criterion(fake_preds, fake_labels)
    
    D_loss = loss_real + loss_fake
    D_loss.backward()
    D_optimizer.step()
    return D_loss.item()

def train_generator(generator, discriminator, G_optimizer, criterion):
    """
    Train the generator for one step. The generator loss is the sum of:
      - an adversarial loss (to fool the discriminator)
      - a convergence loss (to match the ground truth: mean=5, std=1)
    
    Args:
        generator (torch.nn.Module): The generator model.
        discriminator (torch.nn.Module): The discriminator model.
        G_optimizer (torch.optim.Optimizer): Optimizer for the generator.
        criterion: Loss function (BCE) for adversarial loss.
    
    Returns:
        tuple: (generator loss, fake data, sample_mean, sample_std)
    """
    G_optimizer.zero_grad()
    z = torch.randn(batch_size, latent_dim)
    fake_data = generator(z)
    fake_preds = discriminator(fake_data)
    # Adversarial loss: try to fool the discriminator
    real_labels = torch.ones(batch_size, 1)
    adv_loss = criterion(fake_preds, real_labels)
    
    # Convergence loss: force sample statistics to match ground truth (mean=5, std=1)
    sample_mean = torch.mean(fake_data)
    sample_std = torch.std(fake_data)
    conv_loss = (sample_mean - 5)**2 + (sample_std - 1)**2
    
    G_loss = adv_loss + conv_loss
    G_loss.backward()
    G_optimizer.step()
    
    return G_loss.item(), fake_data.detach().numpy(), sample_mean.item(), sample_std.item()

# ---------------------------
# Simulation of Training and Data Storage
# ---------------------------
# Build models and optimizers
generator = build_generator()
discriminator = build_discriminator()
G_optimizer = optim.Adam(generator.parameters(), lr=lr)
D_optimizer = optim.Adam(discriminator.parameters(), lr=lr)
criterion = nn.BCELoss()

# For plotting, define an x-axis and compute the ground truth PDF (constant)
x = np.linspace(0, 10, 1000)
real_pdf = norm.pdf(x, 5, 1)

# Lists to store data for animation
ground_truth_data = []         # Will store real_pdf (constant) per epoch
generator_data = []            # Generator PDF computed from sample statistics
discriminator_data = []        # Discriminator output on x (grid)
losses_G = []                  # Generator losses
losses_D = []                  # Discriminator losses
gen_stats = []                 # (mean, std) from generator

# Pre-store ground truth (same for all epochs)
for _ in range(epochs):
    ground_truth_data.append(real_pdf)
ground_truth_data = np.array(ground_truth_data)

# Training loop
for epoch in range(epochs):
    # Generate real data batch
    real_batch = torch.tensor(true_distribution(batch_size), dtype=torch.float32)
    # Train discriminator
    D_loss = train_discriminator(discriminator, generator, D_optimizer, criterion, real_batch)
    # Train generator
    G_loss, fake_data, sample_mean, sample_std = train_generator(generator, discriminator, G_optimizer, criterion)
    
    losses_D.append(D_loss)
    losses_G.append(G_loss)
    gen_stats.append((sample_mean, sample_std))
    
    # Compute generator PDF using the sample mean and std from this epoch
    gen_pdf = norm.pdf(x, sample_mean, sample_std)
    generator_data.append(gen_pdf)
    
    # Evaluate discriminator on the grid x
    x_tensor = torch.tensor(x.reshape(-1, 1), dtype=torch.float32)
    disc_out = discriminator(x_tensor).detach().numpy().flatten()
    discriminator_data.append(disc_out)
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:03d}/{epochs} | D_loss: {D_loss:.4f} | G_loss: {G_loss:.4f} | "
              f"Gen_Mean: {sample_mean:.4f}, Gen_Std: {sample_std:.4f}")

# Convert lists to arrays for animation
generator_data = np.array(generator_data)
discriminator_data = np.array(discriminator_data)

# ---------------------------
# Animation Functions
# ---------------------------
def create_animation(x, ground_truth_data, generator_data, discriminator_data, losses_G, losses_D, epochs, interval=100):
    """
    Create an animation showing the evolution of the generator and discriminator outputs
    along with loss values.
    
    Args:
        x (np.ndarray): x-axis values.
        ground_truth_data (np.ndarray): Ground truth PDF (epochs x len(x)).
        generator_data (np.ndarray): Generator PDFs (epochs x len(x)).
        discriminator_data (np.ndarray): Discriminator outputs on x (epochs x len(x)).
        losses_G (list): Generator losses per epoch.
        losses_D (list): Discriminator losses per epoch.
        epochs (int): Number of epochs (frames).
        interval (int): Delay between frames in ms.
        
    Returns:
        tuple: (ani, fig) the animation and the figure.
    """
    fig, ax = plt.subplots()
    ax.set_xlim(np.min(x), np.max(x))
    ax.set_ylim(0, 1)
    ax.set_title("GAN Training Progress")
    ax.set_xlabel("x")
    ax.set_ylabel("Probability")
    
    # Plot constant real PDF
    ax.plot(x, ground_truth_data[0], 'k--', label="Real Data (pdata)")
    gen_line, = ax.plot([], [], 'g-', label="Generator (pg)")
    disc_line, = ax.plot([], [], 'b--', label="Discriminator (D)")
    
    # Annotation text for epoch and losses
    info_text = ax.text(0.05, 0.80, "", transform=ax.transAxes)
    ax.legend()
    
    def update(frame):
        gen_line.set_data(x, generator_data[frame])
        disc_line.set_data(x, discriminator_data[frame])
        info_text.set_text(
            f"Epoch: {frame+1}\nG_loss: {losses_G[frame]:.4f}\nD_loss: {losses_D[frame]:.4f}"
        )
        return gen_line, disc_line, info_text
    
    ani = animation.FuncAnimation(fig, update, frames=epochs, interval=interval, blit=True)
    return ani, fig

# Create and display the animation
ani, fig = create_animation(x, ground_truth_data, generator_data, discriminator_data, losses_G, losses_D, epochs, interval=100)
plt.show()

# ---------------------------
# Save the Animation
# ---------------------------
def save_animation(ani, mp4_filename, gif_filename, fps=10):
    """
    Save the animation as MP4 and GIF.
    
    Args:
        ani (FuncAnimation): The animation object.
        mp4_filename (str): Filename for MP4.
        gif_filename (str): Filename for GIF.
        fps (int): Frames per second.
    """
    writer = FFMpegWriter(fps=fps, metadata=dict(artist='Your Name'),
                           extra_args=['-vcodec', 'libx264'])
    ani.save(mp4_filename, writer=writer)
    ani.save(gif_filename, writer='pillow', fps=fps)

# Save the animation in both formats
save_animation(ani, "gan_training_progress.mp4", "gan_training_progress.gif", fps=10)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
from matplotlib.animation import FFMpegWriter
from scipy.stats import multivariate_normal
%matplotlib notebook

# ---------------------------
# Hyperparameters (tuned for better results)
# ---------------------------
latent_dim = 10         # Dimensionality of the latent (noise) vector
hidden_dim = 256        # Increased number of neurons in hidden layers for better capacity
data_dim = 2            # 2D data points
batch_size = 256        # Increased batch size for more stable gradient estimates
epochs = 300            # Increased epochs to allow more training iterations
lr = 0.001              # Increased learning rate for faster convergence

def true_distribution(n):
    """
    Generate samples from the true (real) 2D distribution.
    Args:
        n (int): Number of samples to generate.
    Returns:
        np.ndarray: Array of shape (n, 2) drawn from a multivariate normal
                    distribution with mean [2, -2] and identity covariance.
    """
    mean = [2, -2]
    cov = [[1, 0], [0, 1]]
    return np.random.multivariate_normal(mean, cov, n)

def build_generator():
    """
    Build the generator model using a functional (sequential) approach.
    Returns:
        torch.nn.Sequential: Generator model mapping latent vectors to 2D data points.
    """
    model = nn.Sequential(
        nn.Linear(latent_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, data_dim),
    )
    return model

def train_generator(generator, optimizer):
    """
    Perform a single training step for the generator.
    The generator is trained to maximize the mean of its output, a proxy loss
    that, along with the tuned hyperparameters, helps it better approximate the true distribution.
    Args:
        generator (torch.nn.Module): The generator model.
        optimizer (torch.optim.Optimizer): Optimizer for the generator.
    Returns:
        np.ndarray: Generated fake data from the current training step.
    """
    optimizer.zero_grad()
    # Sample random latent vectors
    z = torch.randn(batch_size, latent_dim)
    fake_data = generator(z)
    # Loss defined as negative mean of generated data (a proxy for improvement)
    loss = -torch.mean(fake_data)
    loss.backward()
    optimizer.step()
    return fake_data.detach().numpy()

def update(frame, fake_scatter_G1, fake_scatter_G2):
    """
    Update function for the animation. This function updates the scatter plot positions 
    for both generator outputs based on the data saved during training.
    Args:
        frame (int): Current frame index.
        fake_scatter_G1 (matplotlib.collections.PathCollection): Scatter plot for generator 1.
        fake_scatter_G2 (matplotlib.collections.PathCollection): Scatter plot for generator 2.
    Returns:
        tuple: Updated scatter plot objects for the animation.
    """
    # Update the offsets of the scatter plots with the generated data for the current frame
    fake_scatter_G1.set_offsets(generated_data_gen1[frame])
    fake_scatter_G2.set_offsets(generated_data_gen2[frame])
    return fake_scatter_G1, fake_scatter_G2

# ---------------------------
# Set the ffmpeg executable path in rcParams
# ---------------------------
mpl.rcParams['animation.ffmpeg_path'] = r'C:\Users\Osvaldo\anaconda3\envs\Oz\Library\bin\ffmpeg.exe'

# ---------------------------
# Direct function calls (no main function)
# ---------------------------

# Build generator models using a functional approach
generator1 = build_generator()
generator2 = build_generator()

# Initialize optimizers for each generator
optimizer1 = optim.Adam(generator1.parameters(), lr=lr)
optimizer2 = optim.Adam(generator2.parameters(), lr=lr)

# Save ground truth data by sampling from the true distribution
num_ground_truth_samples = 1000
ground_truth_data = true_distribution(num_ground_truth_samples)

# Initialize lists to store generated data for each generator
generated_data_gen1 = []
generated_data_gen2 = []

# Training loop: update generators and record their outputs for each epoch
for epoch in range(epochs):
    fake_data1 = train_generator(generator1, optimizer1)
    fake_data2 = train_generator(generator2, optimizer2)
    generated_data_gen1.append(fake_data1)
    generated_data_gen2.append(fake_data2)

# ---------------------------
# Visualization / Animation Setup
# ---------------------------
fig, ax = plt.subplots()
ax.set_xlim(-4, 6)
ax.set_ylim(-6, 2)
ax.set_title("GAN Training Progress")
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")

# Generate grid for the ground truth probability density function (PDF)
x, y = np.mgrid[-4:6:.1, -6:2:.1]
pos = np.dstack((x, y))
rv = multivariate_normal(mean=[2, -2], cov=[[1, 0], [0, 1]])
# Plot the ground truth distribution as a contour plot
ax.contourf(x, y, rv.pdf(pos), cmap='Reds', alpha=0.5)

# Initialize scatter plots for the generators
fake_scatter_G1 = ax.scatter([], [], color='blue', alpha=0.5, label='Generator 1')
fake_scatter_G2 = ax.scatter([], [], color='green', alpha=0.5, label='Generator 2')
ax.legend()

# Create animation; the update function is called for each epoch/frame
ani = animation.FuncAnimation(
    fig,
    lambda frame: update(frame, fake_scatter_G1, fake_scatter_G2),
    frames=len(generated_data_gen1),
    interval=100,
    blit=True
)

# ---------------------------
# Save the animation in MP4 and GIF formats
# ---------------------------
# Create an FFMpegWriter object for MP4 saving
mp4_writer = FFMpegWriter(
    fps=10,
    metadata=dict(artist='Your Name'),
    extra_args=['-vcodec', 'libx264']
)
ani.save('gan_training_progress.mp4', writer=mp4_writer)

# Save as GIF using the pillow writer
ani.save('gan_training_progress.gif', writer='pillow', fps=10)

plt.show()
